# Uebung03: MongoDB

Bringen sie den Datensatz SWISSNAMES3D ( https://www.swisstopo.admin.ch/de/geodata/landscape/names3d.html) in die MongoDB. Das File "swissNAMES3D_PLY.csv" ist bereits im Ordner data zu finden.

Erstellen Sie eine räumliche MONGODB Datenbank mit der Objektklasse: TLM_SIEDLUNGSNAME und der Objektart: Ort

Speichern Sie in der DB:

* NAME
* SPRACHCODE 
* Länge, Breite in WGS84
* Höhe (Gebrauchshöhe LN02, nicht transformiert)


### Vorgehensweise

* **Schritt 1**: Konvertieren Sie die Koordinaten als WGS84 (pyproj) [Beispielcode gegeben]

* **Schritt 2**: Fügen Sie die Daten als GeoJSON in die MongoDB

* **Scrritt 3**: Erstellen Sie folgende Queries: 
   * Welches sind die 20 nächsten Orte von Zürich
   * Welches sind die 10 am höchsten gelegenen Orte in der Schweiz ?
   * Wieviele Orte mit Maximaldistanz 100km gibt es um Basel ?

In [1]:
import pandas as pd

In [14]:
# das dauert... wir laden 200 MB mit geopandas.... mit MongoDB wird das massiv schneller!
# (Man könnte dies auch ohne pandas machen, indem mach einfach durch das CSV File Zeile durch Zeile durchgeht)
df = pd.read_csv("data/swissnames3d/swissNAMES3D_PLY.csv", sep=";", low_memory=False)
df = df[["OBJEKTART", "OBJEKTKLASSE_TLM", "NAME", "SPRACHCODE", "E", "N", "Z"]]

# Wir wollen nur die Objektklasse TLM_SIEDLUNGSNAME:
df = df[df["OBJEKTKLASSE_TLM"] == "TLM_SIEDLUNGSNAME"]
df = df[df["OBJEKTART"] == "Ort"]
df.head()

,OBJEKTART,OBJEKTKLASSE_TLM,NAME,SPRACHCODE,E,N,Z
31444,Ort,TLM_SIEDLUNGSNAME,Egghof,Hochdeutsch inkl. Lokalsprachen,2691205,1294599,624
31445,Ort,TLM_SIEDLUNGSNAME,Regelsrüti,Hochdeutsch inkl. Lokalsprachen,2690625,1294247,630
31446,Ort,TLM_SIEDLUNGSNAME,Chuttler,Hochdeutsch inkl. Lokalsprachen,2692705,1293502,546
31447,Ort,TLM_SIEDLUNGSNAME,Rosegghof,Hochdeutsch inkl. Lokalsprachen,2693481,1292222,509
31448,Ort,TLM_SIEDLUNGSNAME,Eggrüti,Hochdeutsch inkl. Lokalsprachen,2692608,1291656,558


In [15]:
df.shape

(34299, 7)

In [16]:
from pymongo import MongoClient
from getpass import getuser

dbname = "GP2-" + getuser().replace(" ","").lower()
client = MongoClient("db.geopython.xyz",
                     username="igeo",
                     password="m0nG0&dB!")
db = client[dbname]
collection = db.uebung3

In [17]:
from pyproj import Transformer

transformer = Transformer.from_crs(2056, 4326)

# wir können so durch alle Daten durchgehen:
for i in range(0,df.shape[0]):
    name = df.iloc[i]['NAME']
    sprachcode = df.iloc[i]['SPRACHCODE']
    E = float(df.iloc[i]['E'])
    N = float(df.iloc[i]['N'])
    Z = float(df.iloc[i]['Z'])
    
    lat, lng = transformer.transform(E, N)
    
    doc = {"geometry": {
              "type": "Point",
              "coordinates": [lng, lat],
              },
               "name": name,
               "sprachcode": sprachcode,
               "Z": Z   # Die Höhe wird in MongoDB nicht unterstützt
            }
    
    ## und nun dieses doc der MongoDB hinzufügen...
    # ... Code hier...
    record = collection.insert_one(doc)
    
    
    
    ##############################################
    
df = None ## Speicher pandas freigeben    
print(doc)

{'geometry': {'type': 'Point', 'coordinates': [8.088436253165229, 47.04823857971188]}, 'name': 'Landfriede', 'sprachcode': 'Hochdeutsch inkl. Lokalsprachen', 'Z': 709.0, '_id': ObjectId('6462234c4e5e06fd9fe2767b')}



* **Scrritt 3**: Erstellen Sie folgende Queries: 
   * Welches sind die 20 nächsten Orte von Zürich

In [48]:
import geopy.distance

zh = [8.537086, 47.382032]

query = {
    "geometry": {
        "$nearSphere": {
            "$geometry": {
                "type": "Point",
                "coordinates": zh
            }
        }
    }
}

for idx, item in enumerate(collection.find(query).limit(20)):
    doc_coordinates = item['geometry']['coordinates']
    doc_location = (doc_coordinates[1], doc_coordinates[0])  # GeoPy uses (latitude, longitude) format
    distance = geopy.distance.distance(zh[::-1], doc_location).meters  # Calculate distance in meters
    name = item['name']
    print(f"{idx + 1} Name: {name}, Distance: {distance} meters")


1 Name: Zürich, Distance: 1284.4558488714429 meters
2 Name: Zurigo, Distance: 1284.4558488714429 meters
3 Name: Turitg, Distance: 1284.4558488714429 meters
4 Name: Zurich, Distance: 1284.4558488714429 meters
5 Name: Waidbad, Distance: 2833.026811845446 meters
6 Name: Dreiwiesen, Distance: 3111.0737523416656 meters
7 Name: Vorderer Adlisberg, Distance: 3546.597656459846 meters
8 Name: Degenried, Distance: 3569.32392135601 meters
9 Name: Tobelhof, Distance: 3705.5529940264933 meters
10 Name: Hinterer Adlisberg, Distance: 4132.985690410834 meters
11 Name: Gockhausen, Distance: 4653.807803423565 meters
12 Name: Glattpark (Opfikon), Distance: 4766.540359459146 meters
13 Name: Uetliberg, Distance: 4924.324862971358 meters
14 Name: Reckenholz, Distance: 5402.137310580601 meters
15 Name: Zollikon, Distance: 5443.630401786525 meters
16 Name: Wallisellen, Distance: 5452.091015576758 meters
17 Name: Riedenholz, Distance: 5540.190402080072 meters
18 Name: Neugut, Distance: 5601.793113907921 meters


   * Welches sind die 10 am höchsten gelegenen Orte in der Schweiz ?

In [55]:
query = collection.find().sort('Z', -1).limit(10)

for place in query:
    print(place['name'], place['Z'])

Hohsaas 3104.0
Weissfluhjoch 2666.0
Witenwasserntal 2569.0
Bella Lui 2514.0
Col du Gd-St-Bernard 2468.0
Flüelapass 2382.0
Strelapass 2349.0
Val Fenga 2264.0
Ospizio Bernina 2261.0
La Veduta 2233.0



   * Wieviele Orte mit Maximaldistanz 100km gibt es um Basel ?

In [75]:
for item in collection.find({"name": {"$eq": "Basel"}}):
    Basel = item["geometry"]["coordinates"]

In [90]:

query = {
    "geometry": {
        "$nearSphere": {
            "$geometry": {
                "type": "Point",
                "coordinates": Basel
            },
            "$maxDistance": 100 * 1000
        }
    }
}

Test = collection.find(query)
print(len(list(Test)))

20722
